In [1]:
import librosa
import os
import numpy as np
import sys
from dtw import dtw
from numpy.linalg import norm
from numpy import array
import pyaudio
import wave

import heapq

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



**|Heap| = 80**  
**n_mfcc = 20**  
**DTWSeq = mfcc**  
**Window-MinMax**

In [2]:
def initialCorpus(path):
    # 音乐库位置
    audioList = os.listdir(path)

    raw_audioList = {}
    beat_database = {}

    for tmp in audioList:
        audioName = os.path.join(path, tmp)
        if audioName.endswith('.wav'):
            # 读入一维音频序列
            y, sr = librosa.load(audioName)
            # 提取 MFCC 特征
            f = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
            # 存入数据表
            beat_database[audioName] = f

    # 保存音乐节奏数据库
    np.save('beatDatabase_mfcc_20.npy', beat_database)
    
    return beat_database

In [3]:
def readCorpus(path):
    
    # 读入音乐节奏数据库
    all_data = np.load(path, allow_pickle=True)
    beat_database = all_data.item()
    
    return beat_database

In [4]:
def updateCorpus(path, dbpath):
    
    # 音乐库位置
    audioList = os.listdir(path)
    
    # 已保存序列的文件
    raw_db = readCorpus(dbPath)
    raw_files = raw_db.keys()
    
    for tmp in audioList:
        audioName = os.path.join(path, tmp)
        if audioName.endswith('.wav') and audioName not in raw_files:
            y, sr = librosa.load(audioName)
            # 提取 MFCC 特征
            f = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
            # 存入数据表
            beat_database[audioName] = f

    # 保存音乐节奏数据库
    np.save(dbpath, beat_database)

In [5]:
def normlize(data):
    n_mean = np.mean(data, axis=0)
    n_std  = np.std(data, axis=0)
    
    norm_data = np.divide(np.subtract(data, n_mean), n_std)
    return norm_data

In [6]:
def voiceCompare_quick(dbPath, tPath):
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()

    # 读入要识别的录音
    y, sr = librosa.load(tPath)

    # 识别录音的节奏序列
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    beat_frames = librosa.feature.delta(beat_frames,mode ='nearest')
    x = array(beat_frames).reshape(-1, 1)

    # 将待识别的录音序列与语料库中语音逐一做DTW对比
    compare_result = {}
    
    for songID in beat_database.keys():
        y = beat_database[songID]
        y = array(y).reshape(-1, 1)
        
        dist = dtw(x, y).distance
        # print('两段话的差异程度为： ', songID.split("\\")[1], ": ", dist)
        
        compare_result[songID] = dist

    matched_song = min(compare_result, key=compare_result.get)
    print("最接近的录音是：", matched_song)

In [11]:
from sklearn import preprocessing

def voiceCompare(dbPath, tPath):
    # 最大检索数
    aimNum = 80
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()

    # ==== 读入要识别的录音 ====
    y, sr = librosa.load(tPath)

    # 提取录音的 MFCC 特征
    x = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)  # n1 * 20
    
    # 标准化
    for i in range(0, 20):
        x[i] = preprocessing.minmax_scale(x[i])
        # x[i] = normlize(x[i])
    
    x = x.T
    lenx = len(x)

    # ==== 将待识别的录音序列与语料库中语音逐一做DTW对比 ====
    
    # heap for [dist, 时间段，文件名]
    heap = []
    heapq.heapify(heap)  
    
    for songID in beat_database.keys():
        # 取出文件名对应的 mfcc 序列
        y = beat_database[songID]
        
        # 标准化
        for i in range(0, 20):
            y[i] = preprocessing.minmax_scale(y[i])
            # y[i] = normlize(y[i])
            
        y = y.T    
        leny = len(y) # n2 * 20 
        print(leny)

        for tp in range(0, leny - lenx):
            # *加速* 设定距离上限
            full = False  # 堆是否已满
            dist_UB = -10000  # DTW 距离上限
            overBound = False  # 是否过限
            
            if (len(heap) >= aimNum):
                full = True
                dist_UB = -heap[0][0]  # heap top (biggest) DTW dist as UB  
                
            # 计算 DTW(y[tp : tp + lenx])
            total_dist = 0
            
            for i in range(0, lenx):
                # DTW dist
                total_dist += dtw(x[i], y[tp + i], distance_only=False).distance
                
                # *加速* 超过上限直接取消
                if (full and total_dist > dist_UB):
                    overBound = True
                    break
            
            # *加速* 超过上限
            if (overBound):
                continue
            
            # 入栈
            tupleY = (-total_dist, tp, songID) # dtw 距离加负数转为大根堆
            
            heapq.heappush(heap, tupleY)
            if (len(heap) > aimNum):
                heapq.heappop(heap)
            
            print(tupleY)
            
        # end for
        
        # 处理同名短间隔问题
        
        
    return heap

In [8]:
def getTimePoint_dense(dbPath, tPath, vheap):
    res_num = 20 # 定义取出前 res_num 位的结果作为识别结果
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()
    
    # 得到要识别的录音时长
    tTime = librosa.get_duration(filename=tPath)
    
    # 提取前 res_num 个相似的片段并输出对应时间段
    similar_n = heapq.nlargest(res_num, vheap)
    
    print("开始输出相似片段：")
    
    for i in range(0, res_num):
        music_name = similar_n[i][2]  # 录音文件名
        music_time = librosa.get_duration(filename=music_name)  # 录音时长
        
        music_pos = similar_n[i][1]  # 时间段所在帧数
        music_all = len(beat_database[music_name][0])  # 录音总帧数

        frag_st = music_time / music_all * music_pos  # 时间段起点
        frag_en = frag_st + tTime  # 时间段终点
        
        # print(music_name, music_time, music_pos, music_all, frag_st)
        # print("相似度第", i + 1, "位的为文件 ", music_name, "的 ", '%.2f' % frag_st, "到", '%.2f' % frag_en, "秒")
        
        print(music_name, ",", '%.2f' % frag_st, "秒,", '%.2f' % frag_en, "秒")


In [15]:
def getTimePoint(dbPath, tPath, vheap):
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()
    
    # 得到要识别的录音时长
    tTime = librosa.get_duration(filename=tPath)
    
    heapq.nlargest(20, vheap)
    
    # ====== 对 vheap 进行去重 ======
    # 取出文件名
    name_set = set()
    for tp in vheap:
        name_set.add(tp[2])
    # print(name_set)
    
    # 合并下标差小于5的片段
    sheap = []
    for name in name_set:
        # 按下标排序
        nList = [x for x in vheap if x[2] == name]
        sortL = sorted(nList, key=lambda t:t[1])
        
        # 去重
        for tp in sortL:
            if len(sheap) < 1 or sheap[-1][2] != name or abs(sheap[-1][1] - tp[1]) > 5:
                sheap.append(tp)
            else:  
                if (sheap[-1][0] < tp[0]): 
                    sheap[-1] = tp  # 保留距离较小项

    # print(sheap)
    # 提取相似片段并输出对应时间段
    heapq.heapify(sheap)
    if (len(sheap) > 10):
        sheap = heapq.nlargest (10, sheap)
        
    similar_n = sheap
        
    print("开始输出相似片段：")
    
    for i in range(0, len(sheap)):
        music_name = similar_n[i][2]  # 录音文件名
        music_time = librosa.get_duration(filename=music_name)  # 录音时长
        
        music_pos = similar_n[i][1]  # 时间段所在帧数
        music_all = len(beat_database[music_name][0])  # 录音总帧数
        
        frag_st = music_time / music_all * music_pos  # 时间段起点
        frag_en = frag_st + tTime  # 时间段终点
        # print(music_name, music_time, music_pos, "/", music_all, "=", frag_st)
        
        # print(music_name, music_time, music_pos, music_all, frag_st)
        # print("相似度第", i + 1, "位的为文件 ", music_name, "的 ", '%.2f' % frag_st, "到", '%.2f' % frag_en, "秒")
        
        print(music_name, ",", '%.2f' % frag_st, "秒,", '%.2f' % frag_en, "秒")

In [10]:
# 语料库路径
corpus_path = './corpus'

# 数据表路径
dbPath = './beatDatabase_mfcc_20.npy';

# test file path
# testPath = './input/00415250-前5s.wav'
# testPath = './input/00429126-53s_60s.wav'
testPath = './input/00430105-hou5s.wav'

In [13]:
# 1 初始化语料序列库
# beatDB = initialCorpus(corpus_path)

# 2 更新语料库中新音乐文件的序列
# updateCorpus(corpus_path, dbPath)

# 3 读入语料序列库
# beat_database = readCorpus(dbPath)
testPath = './input/00430105-hou5s.wav'
vheap = voiceCompare(dbPath, testPath)

3876
(-1552.395266573545, 0, './corpus\\00415250.wav')
(-1573.9153801972461, 1, './corpus\\00415250.wav')
(-1590.4417497285358, 2, './corpus\\00415250.wav')
(-1606.343303674268, 3, './corpus\\00415250.wav')
(-1617.040728059732, 4, './corpus\\00415250.wav')
(-1623.5257282499406, 5, './corpus\\00415250.wav')
(-1641.580820257328, 6, './corpus\\00415250.wav')
(-1657.9537389136567, 7, './corpus\\00415250.wav')
(-1668.6474987713336, 8, './corpus\\00415250.wav')
(-1691.8583879553662, 9, './corpus\\00415250.wav')
(-1709.761838433118, 10, './corpus\\00415250.wav')
(-1740.8867901854865, 11, './corpus\\00415250.wav')
(-1758.3295897136159, 12, './corpus\\00415250.wav')
(-1779.7628959873873, 13, './corpus\\00415250.wav')
(-1805.2332557277005, 14, './corpus\\00415250.wav')
(-1828.3066111823027, 15, './corpus\\00415250.wav')
(-1841.5009139474896, 16, './corpus\\00415250.wav')
(-1848.2566539900326, 17, './corpus\\00415250.wav')
(-1859.9433391462262, 18, './corpus\\00415250.wav')
(-1867.2553972634669, 

(-1481.8394642039384, 658, './corpus\\00415250.wav')
(-1473.3872183617386, 659, './corpus\\00415250.wav')
(-1477.1145144032962, 660, './corpus\\00415250.wav')
(-1469.52610157269, 661, './corpus\\00415250.wav')
(-1456.5905465847063, 662, './corpus\\00415250.wav')
(-1451.065459388792, 663, './corpus\\00415250.wav')
(-1451.562275689812, 664, './corpus\\00415250.wav')
(-1453.8889216268617, 665, './corpus\\00415250.wav')
(-1459.9238681746829, 666, './corpus\\00415250.wav')
(-1478.9981415805123, 667, './corpus\\00415250.wav')
(-1501.6192561544194, 668, './corpus\\00415250.wav')
(-1505.378413483082, 669, './corpus\\00415250.wav')
(-1504.3197622237967, 695, './corpus\\00415250.wav')
(-1500.1697337339465, 697, './corpus\\00415250.wav')
(-1491.6621207412572, 698, './corpus\\00415250.wav')
(-1494.6413725950006, 704, './corpus\\00415250.wav')
(-1481.6684262984963, 705, './corpus\\00415250.wav')
(-1457.859010512755, 706, './corpus\\00415250.wav')
(-1448.5637816496783, 707, './corpus\\00415250.wav')

(-1395.8822731899745, 1711, './corpus\\00415250.wav')
(-1395.651766736777, 1712, './corpus\\00415250.wav')
(-1395.9839706018283, 1713, './corpus\\00415250.wav')
(-1396.0630696981862, 1714, './corpus\\00415250.wav')
(-1396.0173698484614, 1733, './corpus\\00415250.wav')
(-1395.8341447978928, 1734, './corpus\\00415250.wav')
(-1396.1536367654517, 1735, './corpus\\00415250.wav')
(-1396.215665568383, 1736, './corpus\\00415250.wav')
(-1396.215665568383, 1737, './corpus\\00415250.wav')
(-1396.215665568383, 1738, './corpus\\00415250.wav')
(-1396.215665568383, 1739, './corpus\\00415250.wav')
(-1396.215665568383, 1740, './corpus\\00415250.wav')
(-1396.215665568383, 1741, './corpus\\00415250.wav')
(-1396.215665568383, 1742, './corpus\\00415250.wav')
(-1396.215665568383, 1743, './corpus\\00415250.wav')
(-1396.215665568383, 1744, './corpus\\00415250.wav')
(-1396.215665568383, 1745, './corpus\\00415250.wav')
(-1396.215665568383, 1746, './corpus\\00415250.wav')
(-1396.215665568383, 1747, './corpus\\00

10164
7150
(-1371.1264477989062, 911, './corpus\\00430105.wav')


In [14]:
getTimePoint(dbPath, testPath, vheap)

开始输出相似片段：
./corpus\00415250.wav , 26.35 秒, 28.42 秒
./corpus\00415250.wav , 76.56 秒, 78.62 秒
./corpus\00430105.wav , 21.15 秒, 23.22 秒
./corpus\00429239.wav , 12.24 秒, 14.31 秒
./corpus\00429239.wav , 102.35 秒, 104.42 秒
./corpus\00429239.wav , 207.21 秒, 209.28 秒
./corpus\00429239.wav , 207.35 秒, 209.42 秒
./corpus\00429239.wav , 226.48 秒, 228.55 秒
./corpus\00429239.wav , 226.62 秒, 228.69 秒
./corpus\00429126.wav , 80.27 秒, 82.34 秒
./corpus\00429126.wav , 94.73 秒, 96.80 秒


In [ ]:
inPara = sys.argv

if (len(inPara) < 2):
    print("请输入待识别录音文件路径！")
else:
    if (len(inPara) > 2):
        print("给定语料库路径为:", sys.argv[2])
        corpus_path = sys.argv[2]
    else:
        print("默认语料库路径为：", corpus_path)
    
    if (len(inPara) > 3):
        print("给定数据表路径为:", sys.argv[3])
        dbPath = sys.argv[3]
    else:
        print("默认数据表路径为：", dbPath)
    
    testPath = sys.argv[1]
    vheap = voiceCompare(dbPath, testPath)
    getTimePoint(dbPath, testPath, vheap)
  

In [12]:
testPath2 = './input/00430105-hou5s.wav'
vheap2 = voiceCompare(dbPath, testPath2)
getTimePoint(dbPath, testPath2, vheap2)

3876
(-479.5739887058735, 0, './corpus\\00415250.wav')
(-478.8359127268195, 1, './corpus\\00415250.wav')
(-479.90067633613944, 2, './corpus\\00415250.wav')
(-481.2924239560962, 3, './corpus\\00415250.wav')
(-481.4842509776354, 4, './corpus\\00415250.wav')
(-480.5962525382638, 5, './corpus\\00415250.wav')
(-481.10502410680056, 6, './corpus\\00415250.wav')
(-481.6134977713227, 7, './corpus\\00415250.wav')
(-482.53090477734804, 8, './corpus\\00415250.wav')
(-483.77399249374866, 9, './corpus\\00415250.wav')
(-484.31166733801365, 10, './corpus\\00415250.wav')
(-486.95738607645035, 11, './corpus\\00415250.wav')
(-486.54270680993795, 12, './corpus\\00415250.wav')
(-487.483578376472, 13, './corpus\\00415250.wav')
(-485.9326030090451, 14, './corpus\\00415250.wav')
(-486.74262200295925, 15, './corpus\\00415250.wav')
(-487.95961052924395, 16, './corpus\\00415250.wav')
(-487.9801718592644, 17, './corpus\\00415250.wav')
(-488.25053407996893, 18, './corpus\\00415250.wav')
(-488.891846857965, 19, './

(-444.0209492109716, 164, './corpus\\00415250.wav')
(-444.7062832452357, 165, './corpus\\00415250.wav')
(-445.0355458520353, 166, './corpus\\00415250.wav')
(-447.7565952949226, 167, './corpus\\00415250.wav')
(-451.2093938998878, 168, './corpus\\00415250.wav')
(-448.65624417737126, 169, './corpus\\00415250.wav')
(-448.8663149140775, 170, './corpus\\00415250.wav')
(-448.9028553105891, 171, './corpus\\00415250.wav')
(-450.49098547920585, 172, './corpus\\00415250.wav')
(-448.2081885561347, 173, './corpus\\00415250.wav')
(-450.55474443361163, 174, './corpus\\00415250.wav')
(-451.2813191600144, 175, './corpus\\00415250.wav')
(-452.22529601305723, 176, './corpus\\00415250.wav')
(-452.0399499386549, 177, './corpus\\00415250.wav')
(-450.1728896629065, 180, './corpus\\00415250.wav')
(-450.3114282004535, 185, './corpus\\00415250.wav')
(-450.8878764398396, 186, './corpus\\00415250.wav')
(-448.679741198197, 193, './corpus\\00415250.wav')
(-445.1819368042052, 194, './corpus\\00415250.wav')
(-445.800

(-426.0885477233678, 2242, './corpus\\00429239.wav')
(-424.38459699228406, 2324, './corpus\\00429239.wav')
(-424.69771921262145, 2325, './corpus\\00429239.wav')
(-426.05412431806326, 2326, './corpus\\00429239.wav')
(-426.9647116474807, 5131, './corpus\\00429239.wav')
(-424.3269113563001, 5132, './corpus\\00429239.wav')
(-423.79994412511587, 5982, './corpus\\00429239.wav')
(-426.0295157507062, 9317, './corpus\\00429239.wav')
(-424.9811795242131, 9318, './corpus\\00429239.wav')
(-425.60930436477065, 9319, './corpus\\00429239.wav')
(-423.757024589926, 9320, './corpus\\00429239.wav')
(-426.1998883970082, 9321, './corpus\\00429239.wav')
(-426.285054218024, 9322, './corpus\\00429239.wav')
(-425.7205310501158, 9325, './corpus\\00429239.wav')
(-424.55739149078727, 9326, './corpus\\00429239.wav')
(-425.91639367491007, 9327, './corpus\\00429239.wav')
(-422.89460687339306, 9328, './corpus\\00429239.wav')
(-424.01743790507317, 9329, './corpus\\00429239.wav')
(-423.0458586104214, 9330, './corpus\\0

(-412.07692438177764, 2581, './corpus\\00430105.wav')
(-410.59563629888, 2582, './corpus\\00430105.wav')
(-416.2597876172513, 2583, './corpus\\00430105.wav')
(-416.6663248371333, 2584, './corpus\\00430105.wav')
(-417.5706724245101, 2585, './corpus\\00430105.wav')
(-418.2641292642802, 2586, './corpus\\00430105.wav')
(-420.6080383080989, 2587, './corpus\\00430105.wav')
(-420.78981542401016, 2592, './corpus\\00430105.wav')
(-418.1861768588424, 2780, './corpus\\00430105.wav')
(-419.34593834355474, 2781, './corpus\\00430105.wav')
(-419.0084067992866, 2783, './corpus\\00430105.wav')
(-419.2731943912804, 2784, './corpus\\00430105.wav')
(-419.7819299977273, 2866, './corpus\\00430105.wav')
(-420.1939413920045, 2868, './corpus\\00430105.wav')
(-420.36950275860727, 2878, './corpus\\00430105.wav')
(-418.6746196169406, 2881, './corpus\\00430105.wav')
(-418.5719761159271, 2882, './corpus\\00430105.wav')
(-415.4941132944077, 2883, './corpus\\00430105.wav')
(-419.948793746531, 3206, './corpus\\0043010

In [17]:
getTimePoint(dbPath, testPath2, vheap2)

开始输出相似片段：
./corpus\00430105.wav , 126.93 秒, 129.00 秒
./corpus\00430105.wav , 140.00 秒, 142.07 秒
./corpus\00430105.wav , 59.34 秒, 61.41 秒
./corpus\00430105.wav , 125.70 秒, 127.76 秒
./corpus\00430105.wav , 109.82 秒, 111.88 秒
./corpus\00430105.wav , 59.50 秒, 61.57 秒
./corpus\00430105.wav , 59.95 秒, 62.02 秒
./corpus\00430105.wav , 139.72 秒, 141.79 秒
./corpus\00430105.wav , 78.59 秒, 80.66 秒
./corpus\00430105.wav , 140.28 秒, 142.35 秒
